### Problem Statement
Using PyTorch, set up the pre-trained
network ResNet50. Obtain 10 of your own images that are similar to ImageNet classes
and classify them. Choose 10 images from 5 different classes (2 images per class).
Report the confusion matrix, the accuracy, the f-score, precision and recall of your
classifier. There should be 6 classes representing the 5 classes that your images belong
to as well as an 6th ’other’ class.

### Load required modules

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import glob
import ntpath
import sklearn.metrics
from PIL import Image

In [ ]:
# Load pretrained ResNet50 model
# https://pytorch.org/docs/stable/torchvision/models.html
# Tip: When loading model make sure pretrain argument set to True
# Tip: Good resource for PyTorch projects: https://github.com/pytorch/examples/blob/master/mnist/main.py

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # use gpu device
resnet50 = models.wide_resnet50_2(pretrained=True)
model = resnet50 # load model from torchvision.models
model = model.to(device)  # model operations are sent to GPU


Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

path = '/content/drive/My Drive/Colab Notebooks/RCV/Images/*'

/content/drive/My Drive/Colab Notebooks/RCV/Images/*


In [ ]:
# Create custom Dataset for your images
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.dataset = []
        self.true_vals = []
        self.root_dir = root_dir
        classes = glob.glob(root_dir)
        label_dict = {"Ost1.jpg":9,
                "Ost2.jpg":9,
                "HH1.jpg":4,
                "HH2.jpg":4,
                "HV1.jpg":66,
                "HV2.jpg":66,
                "Peacock1.jpg":84,
                "Peacock2.jpg":84,
                "G1.jpg":366,
                "G2.jpg":366}
      
        for c in classes:
            self.dataset += [[c, label_dict[ntpath.basename(c)]]]
            self.true_vals += [label_dict[ntpath.basename(c)]]
        print(self.true_vals)
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        image_path, target = self.dataset[index]
        image = Image.open(image_path)
        image = transforms.ToTensor()(image)
        ''' Dont need this because transform is None
        if self.transform:
            image = self.transform(image)
       '''
        return image, target

In [ ]:
# Create a DataLoader for your dataset
root_dir = path
transform = None
custom_dataset = CustomDataset(root_dir, transform)
loader = DataLoader(custom_dataset, batch_size=1)

[9, 9, 4, 4, 66, 66, 84, 84, 366, 366]


In [ ]:
# Cycle through custom dataset and pass data into the model
model.eval()
loss = 0
correct = 0
pred_vals = []
class_vals = [9, 4, 66, 84, 366, 1000]
#other = 1000
target_vals = []
with torch.no_grad():
  for (image, target) in loader:
      image = image.to(device)
      target = target.to(device)
      target_vals += [target.cpu().numpy().item()]


      prediction = model(image)
      loss += torch.nn.functional.nll_loss(prediction, target, reduction = 'sum').item()
      pred = prediction.argmax(dim=1, keepdim = True)
      correct += pred.eq(target.view_as(pred)).sum().item()
      index = prediction.cpu().data.numpy().argmax()
      if index in class_vals:
        pred_vals += [index]
      else:
        pred_vals += [1000]
  accuracy = 100.* correct / len(loader.dataset)
  confusion = sklearn.metrics.confusion_matrix(target_vals, pred_vals, labels = class_vals)
  print("Accuracy: ",accuracy, "%")
I 

  precision = []
  recall = []
  fscore = []
  for i in range(0, len(class_vals)):

    recall += [confusion[i,i] / sum(confusion[:,i])]
    if np.isnan([confusion[i,i] / sum(confusion[i,:])]):
      precision += [0]
    else:
      precision += [confusion[i,i] / sum(confusion[i,:])]


    if np.isnan((precision[i] * recall[i]) * 2/ (precision[i] + recall[i])):
      fscore += [0]
    else:
      fscore += [(precision[i] * recall[i]) * 2/ (precision[i] + recall[i])]

  confusion = np.insert(confusion, 0, [9,4,66,84,366,1000], axis = 1)
  confusion = np.insert(confusion, 0, [-1,9,4,66,84,366,1000], axis = 0)
  print("           Predicted Values")
  print(confusion)
  print("       Target Values on the left")  


  for i in range(0, len(class_vals)):
    print("Class Number: ", class_vals[i], "has Precision : ", precision[i], "and has recall: ", recall[i], "and F1 score :", fscore[i])

Accuracy:  70.0 %
           Predicted Values
[[  -1    9    4   66   84  366 1000]
 [   9    2    0    0    0    0    0]
 [   4    0    1    0    0    0    1]
 [  66    0    0    1    0    0    1]
 [  84    0    0    0    2    0    0]
 [ 366    0    0    0    0    1    1]
 [1000    0    0    0    0    0    0]]
       Target Values on the left
Class Number:  9 has Precision :  1.0 and has recall:  1.0 and F1 score : 1.0
Class Number:  4 has Precision :  0.5 and has recall:  1.0 and F1 score : 0.6666666666666666
Class Number:  66 has Precision :  0.5 and has recall:  1.0 and F1 score : 0.6666666666666666
Class Number:  84 has Precision :  1.0 and has recall:  1.0 and F1 score : 1.0
Class Number:  366 has Precision :  0.5 and has recall:  1.0 and F1 score : 0.6666666666666666
Class Number:  1000 has Precision :  0 and has recall:  0.0 and F1 score : 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
